In [1]:
import pandas as pd
import numpy as np
import re
import scipy.sparse as sps
import codecs
from scipy.sparse import identity,coo_matrix, hstack
from collections import Counter
from hybrid_recommender import train_hybrid_recommender
import config

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
# Finds unique fields in a column of pandas dataframe
def process_and_find_unique_fields(dataframe, column):
    column_uniques = dataframe[column].unique()
    unique_fields = []
    for i in range(0, len(column_uniques)):
        unique_candidate = column_uniques[i]
        if unique_candidate != np.nan and unique_candidate != 'None':
            unique_candidate = str(unique_candidate).replace(' ', '')
            unique_candidate = str(unique_candidate).replace('\\', ',')
            unique_candidate = str(unique_candidate).replace(';', ',')
            fields = str(unique_candidate).split(',')
            for j in range(0, len(fields)):
                if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
                    unique_fields.append(fields[j].lower())
    unique_fields = list(set(unique_fields))
    return unique_fields

# Finds unique fields in CPA_state column of pandas dataframe
def process_and_find_unique_states(dataframe, column):
    column_uniques = dataframe[column].unique()
    unique_fields = []
    for i in range(0, len(column_uniques)):
        unique_candidate = column_uniques[i]
        if unique_candidate != np.nan and unique_candidate != 'None':
            unique_candidate = str(unique_candidate).replace(' ', '')
            unique_candidate = str(unique_candidate).replace('\\', ',')
            fields = str(unique_candidate).split(',')
            for j in range(0, len(fields)):
                if fields[j] !='' and fields[j] !='None'and fields[j] !='nan' and (not ('.' in fields[j])):
                    unique_fields.append(fields[j].lower())
    unique_fields = list(set(unique_fields))
    return unique_fields

# Finds top N keywords in column of pandas dataframe
def get_top_Keywords(keyword_unique, N):
    keyword_set = []
    for topics in keyword_unique:
        topic = str(topics).split(', ')
        for top in topic:
            if top != '':
                keyword_set.append(str(top))
    count = Counter(keyword_set)
    common_list = count.most_common(N)
    keyword_list = [ seq[0] for seq in common_list ]
    return keyword_list

In [3]:
# Reading all relevant files from csv / txt
doc = codecs.open( config.courses_file["file_name"], 'rU', config.courses_file["encoding"])
courses = pd.read_csv(doc, sep = '\t')

doc = codecs.open(  config.product_file["file_name"], 'rU', config.product_file["encoding"])
products = pd.read_csv(doc, sep = '\t')

doc = codecs.open( config.product_order_summary_file["file_name"], 'rU', config.product_order_summary_file["encoding"])
products_order_summary = pd.read_csv(doc, sep = '\t')
products_order_summary = products_order_summary[pd.notnull(products_order_summary['SkuID'])]
products_order_summary = products_order_summary[pd.notnull(products_order_summary['EncryptedRecordID'])]

doc = codecs.open( config.user_enrollments_completions_file["file_name"], 'rU', config.user_enrollments_completions_file['encoding'])
course_completion = pd.read_csv(doc, sep = '\t')
course_completion = course_completion[pd.notnull(course_completion['CourseCode'])]
course_completion = course_completion[pd.notnull(course_completion['EncryptedRecordID'])]

doc = codecs.open( config.user_subscriptions_file["file_name"], 'rU', config.user_subscriptions_file["encoding"])
user_subscription = pd.read_csv(doc, sep = '\t')
user_subscription = user_subscription[pd.notnull(user_subscription['SkuID'])]
user_subscription = user_subscription[pd.notnull(user_subscription['EncryptedUserID'])]

demographics = pd.read_csv( config.user__masterlist_file["file_name"], sep = '\t')

# Finding all unique Courses
courses_club_unique = courses['CourseCode'].unique()

courses_enrolments_club_unique = course_completion['CourseCode'].unique()

all_course_club_unique = np.concatenate((courses_club_unique,courses_enrolments_club_unique))
all_course_club_unique = np.unique(all_course_club_unique)


# Finding all unique Products
products_club_unique = products['SkuID'].unique()

products_order_club_unique = products_order_summary['SkuID'].unique()

all_products_club_unique = np.concatenate((products_club_unique, products_order_club_unique))
all_products_club_unique = np.unique(all_products_club_unique)

#Finding all unique Subscriptions
subscriptions_unique = user_subscription['SkuID'].unique()

# Finding all unique Products and products
course_products_club_unique = np.concatenate((all_course_club_unique, all_products_club_unique, subscriptions_unique))
course_products_club_unique = np.unique(course_products_club_unique)


# Finding all unique Users
users_from_demographics_unique = demographics['EncryptedRecordID'].unique()

users_from_subscription_unique = user_subscription['EncryptedUserID'].unique()

users_from_course_completion_unique = course_completion['EncryptedRecordID'].unique()

users_from_products_order_summary_unique = products_order_summary['EncryptedRecordID'].unique()

users_club = np.concatenate((users_from_demographics_unique,users_from_subscription_unique,users_from_course_completion_unique,users_from_products_order_summary_unique))
users_club_unique = np.unique(users_club)


# Build a dictionary with the user index for all users
d_ui = {}
for i in range(len(users_club_unique)):
    user = users_club_unique[i]
    d_ui[user] = i


# Build a dictionary with the course/product index for all course/product
d_ci = {}
for i in range(len(course_products_club_unique)):
    course = course_products_club_unique[i]
    d_ci[course] = i

# Build a sparse matrix for user-item interaction
Mui_tr = sps.lil_matrix((len(users_club_unique), (len(all_course_club_unique) + len(all_products_club_unique))), dtype=np.int8)

# Now fill Mui_tr with the info from course_completion file
for i in range(len(course_completion)):
    user = course_completion["EncryptedRecordID"].values[i]
    course = course_completion["CourseCode"].values[i]
    ui, ci = d_ui[user], d_ci[course]
    Mui_tr[ui, ci] = 1.0

# Now fill Mui_tr with the info from products_order_summary file
for i in range(len(products_order_summary)):
    user = products_order_summary["EncryptedRecordID"].values[i]
    course = products_order_summary["SkuID"].values[i]
    ui, ci = d_ui[user], d_ci[course]
    Mui_tr[ui, ci] = 1.0

# Now fill Mui_tr with the info from user_subscription file
for i in range(len(user_subscription)):
    user = user_subscription["EncryptedUserID"].values[i]
    course = user_subscription["SkuID"].values[i]
    ui, ci = d_ui[user], d_ci[course]
    Mui_tr[ui, ci] = 1.0

# Process and find unique fields in user demographics
user_unique_cred = process_and_find_unique_fields(demographics, 'Credential')
user_unique_expert = process_and_find_unique_fields(demographics, 'Expertise')
user_unique_interest = process_and_find_unique_fields(demographics, 'Interest')
user_unique_judi = process_and_find_unique_fields(demographics, 'Jurisdiction')
user_unique_state = process_and_find_unique_states(demographics, 'State')

# d_ui has list of users
# Build a dict with the user features in ulist
d_uf = {}
feature_count = 0
for i in range(len(user_unique_cred)):
    feature = user_unique_cred[i]
    d_uf['%s_credential' %feature] = i

feature_count = feature_count + i + 1
for i in range(len(user_unique_state)):
    feature = user_unique_state[i]
    d_uf['%s_state' %feature] = i + feature_count

feature_count = feature_count + i + 1
for i in range(len(user_unique_expert)):
    feature = user_unique_expert[i]
    d_uf['%s_expert' %feature] = i + feature_count

feature_count = feature_count + i + 1
for i in range(len(user_unique_interest)):
    feature = user_unique_interest[i]
    d_uf['%s_interest' %feature] = i + feature_count

feature_count = feature_count + i + 1
for i in range(len(user_unique_judi)):
    feature = user_unique_judi[i]
    d_uf['%s_jurisdiction' %feature] = i + feature_count

# d_ui has list of users
# Build a sparse matrix for user-metadata
Mui_uf = sps.lil_matrix((len(users_club_unique), len(d_uf)), dtype=np.int8)

# Now fill Mui_uf with the info from user features
for i in range(len(demographics)):
    if demographics["EncryptedRecordID"].values[i] == '#Name?':
        i = i+1
    user = demographics["EncryptedRecordID"].values[i]
    ui = d_ui[user]
    credential = demographics['Credential'].values[i]
    credential = str(credential).replace(' ', '')
    credential = str(credential).replace('\\', ',')
    fields = str(credential).split(',')
    for j in range(0, len(fields)):
        if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
            uf = d_uf[fields[j].lower() + '_credential']
            Mui_uf[ui, uf] = 1.0


    state = demographics['State'].values[i]
    state = str(state).replace(' ', '')
    state = str(state).replace('\\', ',')
    fields = str(state).split(',')
    for j in range(0, len(fields)):
        if fields[j] !='' and fields[j] !='None'and fields[j] !='nan' and (not ('.' in fields[j])):
            uf = d_uf[fields[j].lower() + '_state']
            Mui_uf[ui, uf] = 1.0

    expert = demographics['Expertise'].values[i]
    expert = str(expert).replace(' ', '')
    expert = str(expert).replace('\\', ',')
    fields = str(expert).split(',')
    for j in range(0, len(fields)):
        if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
            uf = d_uf[fields[j].lower() + '_expert']
            Mui_uf[ui, uf] = 1.0

    interest = demographics['Interest'].values[i]
    interest = str(interest).replace(' ', '')
    interest = str(interest).replace('\\', ',')
    fields = str(interest).split(',')
    for j in range(0, len(fields)):
        if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
            uf = d_uf[fields[j].lower() + '_interest']
            Mui_uf[ui, uf] = 1.0

    jurisdiction = demographics['Jurisdiction'].values[i]
    jurisdiction = str(jurisdiction).replace(' ', '')
    jurisdiction = str(jurisdiction).replace('\\', ',')
    fields = str(jurisdiction).split(',')
    for j in range(0, len(fields)):
        if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
            uf = d_uf[fields[j].lower() + '_jurisdiction']
            Mui_uf[ui, uf] = 1.0

Mui_uf = hstack([Mui_uf, identity(len(users_club_unique))])

required_columns = ['CourseCode', 'AllFieldOfStudy', 'TotalCreditHours', 'Topic', 'Type', 'PriceRange', 'Keywords', 'EventEndDate', 'EventStartDate']
courses = pd.DataFrame(courses, columns=required_columns)

# Process and find unique fields in Courses
course_unique_FOS = process_and_find_unique_fields(courses, 'AllFieldOfStudy')

set_topics = []
course_topic = courses['Topic'].unique()
for i in range(0, len(course_topic)):
    set_topics.append([x.strip() for x in (str((course_topic[i]))).split('|')])
course_unique_topics = []
for topics in course_topic:
    topic = str(topics).split('|')
    for top in topic:
        if top != '' and top !='None' and top !='nan':
            course_unique_topics.append(str(top))

course_unique_topics = list(set(course_unique_topics))

course_unique_type = list(courses['Type'].unique())
course_unique_type = [x for x in course_unique_type if str(x) != 'nan']

course_unique_keywords = get_top_Keywords(courses['Keywords'].unique(), 100)

course_price = courses['PriceRange'].unique()
average_course_price_unique =[]
for i in range(0, len(course_price)):
    if not (pd.isnull(course_price[i])):
        priceends = course_price[i].split(' - ')
        for j in range(0, len(priceends)):
            priceends[j] = float(priceends[j].replace('$', '').replace(',',''))
        average_course_price_unique.append(int(np.mean(priceends)))

course_bins = np.linspace(0, max(average_course_price_unique), 10)

required_columns = ['ProductCode', 'SkuID', 'Type', 'AllFieldOfStudy', 'TotalCreditHours', 'PriceRange', 'Topic', 'Keywords']
products = pd.DataFrame(products, columns=required_columns)

# Process and find unique fields in Products
product_unique_type = list(products['Type'].unique())
product_unique_type = [x for x in product_unique_type if str(x) != 'nan']

product_unique_FOS = process_and_find_unique_fields(products, 'AllFieldOfStudy')
len(product_unique_FOS)#36

set_topics = []
product_topic = products['Topic'].unique()
for i in range(0, len(product_topic)):
    set_topics.append([x.strip() for x in (str((product_topic[i]))).split('|')])
product_unique_topics = []
for topics in product_topic:
    topic = str(topics).split('|')
    for top in topic:
        if top != '' and top !='None' and top !='nan':
            product_unique_topics.append(str(top))

product_unique_topics = list(set(product_unique_topics))

product_unique_keywords = get_top_Keywords(products['Keywords'].unique(), 100)

product_price = products['PriceRange'].unique()
average_product_price_unique =[]
for i in range(0, len(product_price)):
    if not (pd.isnull(product_price[i])):
        priceends = product_price[i].split(' - ')
        for j in range(0, len(priceends)):
            priceends[j] = float(priceends[j].replace('$', '').replace(',',''))
        average_product_price_unique.append(int(np.mean(priceends)))

product_bins = np.linspace(0, max(average_product_price_unique), 10)

#d_ci has list of courses/products
# Build a dictionary with the item features
d_if = {}
unique_type = list(set().union(product_unique_type,course_unique_type))
feature_count = 0
for i in range(len(unique_type)):
    feature = unique_type[i]
    d_if['%s_type' %feature] = i

unique_FOS = list(set().union(product_unique_FOS,course_unique_FOS))
feature_count = feature_count + i + 1
for i in range(len(unique_FOS)):
    feature = unique_FOS[i]
    d_if['%s_FOS' %feature] = i + feature_count

feature_count = feature_count + i + 1
d_if['TotalCreditHours'] = feature_count

feature_count = feature_count + 1
unique_topic = list(set().union(product_unique_topics,course_unique_topics))
for i in range(len(unique_topic)):
    feature = unique_topic[i]
    d_if['%s_topic' %feature] = i + feature_count

unique_keywords = list(set().union(product_unique_keywords,course_unique_keywords))
feature_count = feature_count + i + 1

for i in range(len(unique_keywords)):
    feature = unique_keywords[i]
    d_if['%s_keyword' %feature] = i + feature_count

price_range = max(max(average_product_price_unique), max(average_course_price_unique))
price_bins = np.linspace(0, price_range, 10)
feature_count = feature_count + i + 1

for i in range(len(product_bins)):
    d_if['price Range %s' %str(i+1)] = i + feature_count
feature_count = feature_count + i + 1

# Build the user x item matrices using scipy lil_matrix
Mui_if = sps.lil_matrix((len(course_products_club_unique), len(d_if)), dtype=np.int8)

# Now fill Mui_tr with the info from course features
for i in range(len(courses)):
    course = courses["CourseCode"].values[i]
    ci = d_ci[course]
    FOS = courses['AllFieldOfStudy'].values[i]
    FOS = str(FOS).replace(' ', '')
    FOS = str(FOS).replace('\\', ',')
    FOS = str(FOS).replace(';', ',')
    fields = str(FOS).split(',')
    for j in range(0, len(fields)):
        if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
            fi = d_if[fields[j].lower() + '_FOS']
            Mui_if[ci, fi] = 1.0

    credit_hours = courses["TotalCreditHours"].values[i]
    if not np.isnan(credit_hours):
        fi = d_if['TotalCreditHours']
        Mui_if[ci, fi] = credit_hours

    typec = courses['Type'].values[i] + '_type'
    fi = d_if[typec]
    Mui_if[ci, fi] = 1.0

    pricerange = courses['PriceRange'].values[i]
    if (not (pd.isnull(pricerange))):
        priceends = str(pricerange).split(' - ')
        for j in range(0, len(priceends)):
            priceends[j] = float(priceends[j].replace('$', '').replace(',',''))
        price_avg = int(np.mean(priceends))
        course_price_bin = int(np.digitize(price_avg, price_bins))
        pricerange = 'price Range %s' %course_price_bin
        fi = d_if[pricerange]
        Mui_if[ci, fi] = 1.0

    topic = str(courses['Topic'].values[0]).split('|')
    for top in topic:
        if top != '' and top !='None' and top !='nan':
            fi = d_if[str(top) + '_topic']
            Mui_if[ci, fi] = 1.0

# Now fill Mui_tr with the info from product features
for i in range(len(products)):
    product = products["SkuID"].values[i]
    ci = d_ci[product]
    FOS = products['AllFieldOfStudy'].values[i]
    FOS = str(FOS).replace(' ', '')
    FOS = str(FOS).replace('\\', ',')
    FOS = str(FOS).replace(';', ',')
    fields = str(FOS).split(',')
    for j in range(0, len(fields)):
        if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
            fi = d_if[fields[j].lower() + '_FOS']
            Mui_if[ci, fi] = 1.0

    credit_hours = products["TotalCreditHours"].values[i]
    if not np.isnan(credit_hours):
        fi = d_if['TotalCreditHours']
        Mui_if[ci, fi] = credit_hours

    typec = products['Type'].values[i]
    if str(typec) != 'nan':
        typec = typec + '_type'
        fi = d_if[typec]
        Mui_if[ci, fi] = 1.0

    pricerange = products['PriceRange'].values[i]
    if (not (pd.isnull(pricerange))):
        priceends = str(pricerange).split(' - ')
        for j in range(0, len(priceends)):
            priceends[j] = float(priceends[j].replace('$', '').replace(',',''))
        price_avg = int(np.mean(priceends))
        product_price_bin = int(np.digitize(price_avg, price_bins))
        pricerange = 'price Range %s' %product_price_bin
        fi = d_if[pricerange]
        Mui_if[ci, fi] = 1.0

    topic = str(products['Topic'].values[0]).split('|')
    for top in topic:
        if top != '' and top !='None' and top !='nan':
            fi = d_if[str(top) + '_topic']
            Mui_if[ci, fi] = 1.0

Mui_if = hstack([Mui_if, identity(len(course_products_club_unique))])

# Filtering out courses with metadata, since only they can be recommended
courses_with_metadata_index = []
courses_with_metadata = []
for i in range(len(all_course_club_unique)):
    if any(courses.CourseCode == all_course_club_unique[i]):
        courses_with_metadata.append(all_course_club_unique[i])
        courses_with_metadata_index.append(d_ci[all_course_club_unique[i]])

courses_with_metadata = np.array(courses_with_metadata)

# Filtering out products with metadata, since only they can be recommended
products_with_metadata_index = []
products_with_metadata = []
for i in range(len(all_products_club_unique)):
    if any(products.SkuID == all_products_club_unique[i]):
        products_with_metadata.append(all_products_club_unique[i])
        products_with_metadata_index.append(d_ci[all_products_club_unique[i]])

products_with_metadata = np.array(products_with_metadata)

dictionary_courses_with_metadata = {}
for item in courses_with_metadata:
        item_info = courses.loc[courses['CourseCode'] == item]
        if len(item_info) > 0:
            FOS = item_info.iloc[-1]['AllFieldOfStudy']
            FOS_array = []
            if FOS != np.nan and FOS != 'None':    
                FOS = str(FOS).replace(';', ',')
                fields = str(FOS).split(', ')
                for j in range(0, len(fields)):
                    if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
                        FOS_array.append(fields[j])
            try:
                start_date = datetime.strptime(item_info.iloc[-1]['EventStartDate'], '%Y-%m-%d %H:%M').isoformat()
            except:
                start_date = None
            try:
                end_date = datetime.strptime(item_info.iloc[-1]['EventStartDate'], '%Y-%m-%d %H:%M').isoformat()
            except:
                end_date = None
            dictionary_courses_with_metadata[item] = {'itemId' : item,
                                                   'Type' : item_info.iloc[-1]['Type'],
                                                   'AllFieldOfStudy' : FOS_array,
                                                   'EventEndDate' : end_date,
                                                   'EventStartDate' : start_date}


dictionary_products_with_metadata = {}
for item in products_with_metadata:
        item_info = products.loc[products['SkuID'] == item]
        len_item_info = len(item_info)
        if len_item_info > 0:
            FOS = item_info.iloc[-1]['AllFieldOfStudy']
            FOS_array = []
            if FOS != np.nan and FOS != 'None':    
                FOS = str(FOS).replace(';', ',')
                fields = str(FOS).split(', ')
                for j in range(0, len(fields)):
                    if fields[j] !='' and fields[j] !='None'and fields[j] !='nan':
                        FOS_array.append(fields[j])
            try:
                start_date = datetime.strptime(item_info.iloc[-1]['EventStartDate'], '%Y-%m-%d %H:%M').isoformat()
            except:
                start_date = None
            try:
                end_date = datetime.strptime(item_info.iloc[-1]['EventStartDate'], '%Y-%m-%d %H:%M').isoformat()
            except:
                end_date = None
            dictionary_products_with_metadata[item] ={'itemId' : item_info.iloc[-1]['ProductCode'],
                                                       'Type' : item_info.iloc[-1]['Type'],
                                                       'AllFieldOfStudy' : FOS_array,
                                                       'EventEndDate' : end_date,
                                                       'EventStartDate' : start_date}



In [29]:
import gc
del model
gc.collect()

0

In [25]:
from skopt import forest_minimize
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import lightfm
import pickle

In [31]:
epochs= 40
learning_rate= 0.00815902860519 
no_components= 100 
item_alpha= 0.000448849033535 
scale= 0.0358901605615    
user_alpha = item_alpha * scale
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=learning_rate,
                no_components=no_components,
                user_alpha=user_alpha,
                item_alpha=item_alpha)
model.fit( Mui_tr, item_features = Mui_if, user_features=Mui_uf,
          epochs=epochs,
          num_threads=4, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39


In [32]:
doc = codecs.open( config.courses_file["file_name"], 'rU', config.courses_file["encoding"])
courses = pd.read_csv(doc, sep = '\t')
doc = codecs.open(  config.product_file["file_name"], 'rU', config.product_file["encoding"])
products = pd.read_csv(doc, sep = '\t')

In [33]:
userid = 'zfePq8N2N87F2UYqWnWGp8bjCVTchxkV1VDVT3A7IBY='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

In [34]:
predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
2: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
5: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
6: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
7: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
8: E6B54DB8DFB74674B43B656BB7F1A206
XBRL Fundamentals: Understanding the US GAAP Taxonomy
-
9: 254834CB6A214FED9F153A898D7E43A7
Governme

[]

In [35]:
userid = 'gdf02qFth7wzK6B3wuXPzg=='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
2: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
5: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
6: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
7: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
8: E6B54DB8DFB74674B43B656BB7F1A206
XBRL Fundamentals: Understanding the US GAAP Taxonomy
-
9: 254834CB6A214FED9F153A898D7E43A7
Governme

[]

In [36]:
userid = 'Fl24VZQE1yXsPxgOTzav0vf8ri5gmdjowE5L0miCg18='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
2: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
5: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
6: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
7: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
8: E6B54DB8DFB74674B43B656BB7F1A206
XBRL Fundamentals: Understanding the US GAAP Taxonomy
-
9: 254834CB6A214FED9F153A898D7E43A7
Governme

[]

In [37]:
userid = 'gD9WJy7hzwuj4RYNGbqtrQASW7a6Xl4m4ejqyakIuCw='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
2: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
5: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
6: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
7: A3B8981C67AE46C28C26A89C5D0EFB3C
Advanced Tax Planning S Corporations (11/10/2017)
-
8: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
9: 3735970EC26149A68936FDE8FAD64490
Multistate I

[]

In [38]:
userid = 'tU0Kx3IWl3Zk2IPVD97kAna8+8Y5D0xun9H3uI9zikg='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
2: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
5: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
6: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
7: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
8: E6B54DB8DFB74674B43B656BB7F1A206
XBRL Fundamentals: Understanding the US GAAP Taxonomy
-
9: 254834CB6A214FED9F153A898D7E43A7
Governme

[]

In [39]:
userid = 'JOv+s7BwRpeMM2k3M6beYDTpnObMsHjmO9fgNtnHhTw='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
2: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
5: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
6: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
7: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
8: E6B54DB8DFB74674B43B656BB7F1A206
XBRL Fundamentals: Understanding the US GAAP Taxonomy
-
9: 254834CB6A214FED9F153A898D7E43A7
Governme

[]

In [40]:
userid = 'sj46lvYsHdxsd1LYZEymsHxMLwToEbijyfVLdGgKbgE='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
2: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
5: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
6: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
7: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
8: E6B54DB8DFB74674B43B656BB7F1A206
XBRL Fundamentals: Understanding the US GAAP Taxonomy
-
9: 254834CB6A214FED9F153A898D7E43A7
Governme

[]

In [41]:
userid = 'L2AwjZ0dAAw+IxGoiinennSISwNBtCBGksevpUhcVWk='
user_course_int = course_completion.loc[course_completion['EncryptedRecordID'] == userid]
user_product_int = products_order_summary.loc[products_order_summary['EncryptedRecordID'] == userid]

predict_courses_for_user(userid, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = 10)
predict_products_for_user(userid, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = 10)

Courses
1: 6BD53CA4A3CB4CEBA3B3E00F6401D0A8
Governmental Accounting and Reporting Advisor: Introduction to Fair Value (Yellow Book Compliant)
-
2: 56522D76E01C4708A86ADD9EACAA775D
1040 Tax Return Workshop—Tax Computations: Part II
-
3: 5934A26E1F3C49738F3946BD0EB19D96
Advanced Controller and CFO Skills: Improve Your Leadership Effectiveness by Looking Ahead; Improve Your Change Agent Skills Through Self-Coaching
-
4: AF7F88C3999A40079675123EA34720E7
Real-World Business Ethics: Cloud Brilliance.com – You Are the CFO
-
5: 57488C642D394DA1AD4C5C6054BF6EDC
IT Risks and Controls in Current and Emerging Environments: Cloud and Mobile Computing and Other Current Topics
-
6: 43E00F05A421424AB27594123F68DF3B
Individual Tax Fundamentals (11/13/2017)
-
7: 1075A3B158F847348DE805D8261EEB49
Detecting and Preventing the Top Ten Fraud Schemes: Revenue Fraud Schemes
-
8: E6B54DB8DFB74674B43B656BB7F1A206
XBRL Fundamentals: Understanding the US GAAP Taxonomy
-
9: 254834CB6A214FED9F153A898D7E43A7
Governme

[]

In [42]:
def predict_courses_for_user(user, model, d_ui, courses_with_metadata_index, courses_with_metadata, dictionary_courses_with_metadata, user_course_int,courses, top_N = -1):
    # Get index of user from user dictionary
    try:
        user_id = d_ui[user]
        
    except Exception as e:
        logging.error('User ID does not exist')
        raise Exception("User ID does not exist")
    
    # Predict Scores for all courses for this user and store in dictionary
    scores = model.predict(user_id, np.array(courses_with_metadata_index))
    d_scores = {}
    for i in range(len(scores)):
        score = scores[i]
        d_scores[courses_with_metadata[i]] = score
        
    # Sorting scores in order
    top_items = courses_with_metadata[np.argsort(-scores)]
    
    # Iterating through all courses and populating the list of top N items and their metadata
    top_N_list = []
    counter = 0
    print 'Courses'
    for item in top_items:
        not_courses = user_course_int.loc[user_course_int['CourseCode'] == item]
        if len(not_courses) == 0:
            print str(counter+1) + ': ' + str(item)
            title_db = courses.loc[courses['CourseCode'] == item]
            print title_db.iloc[-1]['Title']
            print '-'
            counter =counter + 1
            
        if (top_N != -1) and (counter == top_N):
            break

In [14]:
def predict_products_for_user(user, model, d_ui, products_with_metadata_index, products_with_metadata, dictionary_products_with_metadata,user_product_int,products, top_N = -1):
    # Get index of user from user dictionary
    try:
        user_id = d_ui[user]
        
    except Exception as e:
        logging.error('User ID does not exist')
        raise Exception("User ID does not exist")
    
    # Predict Scores for all products for this user and store in dictionary
    scores = model.predict(user_id, np.array(products_with_metadata_index))
    d_scores = {}
    for i in range(len(scores)):
        score = scores[i]
        d_scores[products_with_metadata[i]] = score
    top_items = products_with_metadata[np.argsort(-scores)]
    
    # Iterating through all products and populating the list of top N items and their metadata
    top_N_list = []
    counter = 0
    print 'Products'
    for item in top_items:
        not_products = user_product_int.loc[user_product_int['SkuID'] == item]
        if len(not_products) == 0:
            prodwithsku = products.loc[products['SkuID'] == item]
            prodcode = prodwithsku.iloc[-1]['ProductCode']
            prodtit = prodwithsku.iloc[-1]['Title']
            print str(counter+1) + ': ' + prodcode
            print prodtit
            print '-'
            counter =counter + 1
            if (counter == top_N) and (top_N != -1):
                break
    return top_N_list


#### 1
Maximimum p@k found: 0.19531

Optimal parameters:

epochs: 68

learning_rate: 0.0271016262622

no_components: 59

item_alpha: 0.000114232541556

scaling: 0.0505980387466







#### 2
epochs: 36

learning_rate: 0.0303690277085

no_components: 88

item_alpha: 0.000154468627875

scale: 0.0192719675083

Time taken: 2331.9187

Function value obtained: -0.2019


#### 3
epochs: 28

learning_rate: 0.0386129433916

no_components: 122

item_alpha: 7.88850830743e-05

scale: 0.00890053497733

Iteration No: 3 ended. Evaluation done at random point.

Time taken: 3023.3377

Function value obtained: -0.2833

Iteration No: 1 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
epochs: 36
learning_rate: 0.0303690277085
no_components: 88
item_alpha: 0.000154468627875
scale: 0.0192719675083
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2331.9187
Function value obtained: -0.2019
Current minimum: -0.2019
Iteration No: 2 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
epochs: 99
learning_rate: 0.291649845477
no_components: 109
item_alpha: 7.4399773602e-05
scale: 0.151736030516
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 5087.5951
Function value obtained: -0.0001
Current minimum: -0.2019
Iteration No: 3 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
epochs: 11
learning_rate: 0.881877226297
no_components: 175
item_alpha: 0.00017740928056
scale: 0.00949837261602
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 4206.9628
Function value obtained: -0.0001
Current minimum: -0.2019
Iteration No: 4 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
epochs: 33
learning_rate: 0.00814854262879
no_components: 27
item_alpha: 1.76434940795e-05
scale: 0.00364469921407
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1087.3056
Function value obtained: -0.0859
Current minimum: -0.2019
Iteration No: 5 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 111
Epoch 112
Epoch 113
Epoch 114
Epoch 115
epochs: 116
learning_rate: 0.00815902860519
no_components: 179
item_alpha: 0.000448849033535
scale: 0.0358901605615
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 7807.9910
Function value obtained: -0.1026
Current minimum: -0.2019
Iteration No: 6 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
epochs: 71
learning_rate: 0.180752553867
no_components: 105
item_alpha: 7.94637412819e-05
scale: 0.109766783631
Iteration No: 6 ended. Evaluation done at random point.
Time taken: 4145.5749
Function value obtained: -0.0001
Current minimum: -0.2019
Iteration No: 7 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
epochs: 65
learning_rate: 0.748598548823
no_components: 23
item_alpha: 0.000186140771089
scale: 0.00998551452033
Iteration No: 7 ended. Evaluation done at random point.
Time taken: 1174.1516
Function value obtained: -0.0000
Current minimum: -0.2019
Iteration No: 8 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
epochs: 100
learning_rate: 0.188843150862
no_components: 119
item_alpha: 4.55718026116e-05
scale: 0.00120050686884
Iteration No: 8 ended. Evaluation done at random point.
Time taken: 5294.4158
Function value obtained: -0.0001
Current minimum: -0.2019
Iteration No: 9 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36

teration No: 1 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
epochs: 100
learning_rate: 0.00139979092292
no_components: 37
item_alpha: 0.000215388838627
scale: 0.0476021765235
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1916.6598
Function value obtained: -0.0505
Current minimum: -0.0505
Iteration No: 2 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
epochs: 81
learning_rate: 0.00713574903019
no_components: 37
item_alpha: 5.38238622108e-05
scale: 0.00283309916395
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1663.8152
Function value obtained: -0.0948
Current minimum: -0.0948
Iteration No: 3 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
epochs: 28
learning_rate: 0.0227398409163
no_components: 122
item_alpha: 7.88850830743e-05
scale: 0.00890053497733
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3054.0798
Function value obtained: -0.1947
Current minimum: -0.1947
Iteration No: 4 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 111
Epoch 112
Epoch 113
Epoch 114
Epoch 115
Epoch 116
Epoch 117
Epoch 118
Epoch 119
Epoch 120
Epoch 121
Epoch 122
Epoch 123
Epoch 124
Epoch 125
Epoch 126
Epoch 127
Epoch 128
Epoch 129
Epoch 130
Epoch 131
Epoch 132
Epoch 133
Epoch 134
Epoch 135
Epoch 136
Epoch 137
Epoch 138
Epoch 139
Epoch 140
Epoch 141
Epoch 142
Epoch 143
Epoch 144
Epoch 145
Epoch 146
Epoch 147
epochs: 148
learning_rate: 0.00128309838443
no_components: 129
item_alpha: 0.00067691810234
scale: 0.023981466838
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 6815.9836
Function value obtained: -0.0503
Current minimum: -0.1947
Iteration No: 5 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 111
Epoch 112
Epoch 113
Epoch 114
Epoch 115
Epoch 116
Epoch 117
Epoch 118
epochs: 119
learning_rate: 0.00131601711594
no_components: 80
item_alpha: 8.29973918428e-05
scale: 0.0430302262756
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 4204.7592
Function value obtained: -0.0654
Current minimum: -0.1947
Iteration No: 6 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 111
Epoch 112
Epoch 113
Epoch 114
Epoch 115
Epoch 116
Epoch 117
Epoch 118
Epoch 119
Epoch 120
Epoch 121
Epoch 122
Epoch 123
Epoch 124
Epoch 125
Epoch 126
Epoch 127
Epoch 128
Epoch 129
Epoch 130
Epoch 131
Epoch 132
epochs: 133
learning_rate: 0.0174872294366
no_components: 35
item_alpha: 0.000808195938312
scale: 0.0512699765082
Iteration No: 6 ended. Evaluation done at random point.
Time taken: 2884.1922
Function value obtained: -0.0922
Current minimum: -0.1947
Iteration No: 7 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90